In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import time
import tensorflow as tf

# Measure the start time before the process
start_time = time.perf_counter()

# Load the dataset
file_path = '/content/drive/MyDrive/bpm flo/merged_music_dataset(in).csv'
df = pd.read_csv(file_path)

# Step 1: Classify songs by tempo
df['tempo_class'] = df['beats.per.minute'].apply(lambda bpm: 'fast' if bpm >= 100 else 'slow')

# Step 2: Assign emotional feeling labels based on tempo, valance, and energy
def classify_feeling(row):
    valance = row['valance']
    energy = row['energy']
    tempo = row['tempo_class']

    if tempo == 'fast':
        if valance >= 60 and energy >= 60:
            return 'fun, energetic'
        elif valance < 60 and energy >= 60:
            return 'intense, edgy'
        else:
            return 'reflective, moody'
    else:  # slow
        if valance < 40 and energy < 40:
            return 'sad, mellow'
        elif valance >= 60 and 40 <= energy < 60:
            return 'romantic, chill'
        else:
            return 'reflective, moody'

df['feeling'] = df.apply(classify_feeling, axis=1)

# Measure the end time after the process
end_time = time.perf_counter()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print out the runtime type and time taken
runtime_type = "GPU" if tf.config.list_physical_devices('GPU') else "CPU"
print(f"🔍 Current Runtime: {runtime_type}")
print(f"⏱️ Time taken for BPM classification and feeling assignment: {elapsed_time:.6f} seconds")

# Preview the result
print(df[['title', 'artist', 'beats.per.minute', 'tempo_class', 'energy', 'valance', 'feeling']].head())


🔍 Current Runtime: GPU
⏱️ Time taken for BPM classification and feeling assignment: 1.025545 seconds
                    title         artist  beats.per.minute tempo_class  \
0         Blinding Lights     The Weeknd               171        fast   
1        Watermelon Sugar   Harry Styles                95        slow   
2  Mood (feat. iann dior)       24kGoldn                91        slow   
3       Someone You Loved  Lewis Capaldi               110        fast   
4                 Perfect     Ed Sheeran                95        slow   

   energy  valance            feeling  
0    73.0     33.0      intense, edgy  
1    82.0     56.0  reflective, moody  
2    72.0     73.0  reflective, moody  
3    41.0     45.0  reflective, moody  
4    45.0     17.0  reflective, moody  


In [ ]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 126.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.0 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="streamlit")


In [ ]:
%%writefile app.py
import os
import tensorflow as tf
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import streamlit as st

# Function to determine the runtime type and measure its efficiency
def get_colab_runtime():
    if 'COLAB_TPU_ADDR' in os.environ:
        return "TPU"
    elif tf.config.list_physical_devices('GPU'):
        return "GPU"
    else:
        return "CPU"

# Measure the time before running the function
start_time = time.perf_counter()

runtime_type = get_colab_runtime()

# Measure the time after running the function
end_time = time.perf_counter()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Streamlit interface and BPM-based code
st.title("🎶 Tempo-Based Song Classifier & Visualizer")

# Load the dataset
file_path = '/content/drive/MyDrive/bpm flo/merged_music_dataset(in).csv'
df = pd.read_csv(file_path)

# Classify by tempo
df['tempo_class'] = df['beats.per.minute'].apply(lambda bpm: 'fast' if bpm >= 100 else 'slow')

# Add feeling column
def classify_feeling(row):
    valance = row['valance']
    energy = row['energy']
    tempo = row['tempo_class']
    if tempo == 'fast':
        if valance >= 60 and energy >= 60:
            return 'fun, energetic'
        elif valance < 60 and energy >= 60:
            return 'intense, edgy'
        else:
            return 'reflective, moody'
    else:
        if valance < 40 and energy < 40:
            return 'sad, mellow'
        elif valance >= 60 and 40 <= energy < 60:
            return 'romantic, chill'
        else:
            return 'reflective, moody'

df['feeling'] = df.apply(classify_feeling, axis=1)

# Streamlit UI
st.markdown(f"🔍 Current Colab Runtime Type: {runtime_type}")
st.markdown(f"⏱️ Time taken to determine runtime: {elapsed_time:.6f} seconds")

# Add Tempo Class filter
tempo_filter = st.selectbox("Select Tempo Class", ["All", "Fast", "Slow"])

# Add Mood filter
mood_filter = st.selectbox("Select Mood", ["All", "fun, energetic", "intense, edgy", "reflective, moody", "sad, mellow", "romantic, chill"])

# Add search filter
search = st.text_input("Enter keyword").lower()

# Apply the filters based on user selection
filtered_df = df

# Tempo filter
if tempo_filter != "All":
    filtered_df = filtered_df[filtered_df['tempo_class'] == tempo_filter.lower()]

# Mood filter
if mood_filter != "All":
    filtered_df = filtered_df[filtered_df['feeling'] == mood_filter]

# Keyword search filter
if search:
    filtered_df = filtered_df[filtered_df['title'].str.lower().str.contains(search) | filtered_df['artist'].str.lower().str.contains(search)]

# Display the filtered data
if not filtered_df.empty:
    st.success(f"Found {len(filtered_df)} result(s)")
    st.dataframe(filtered_df[['title', 'artist', 'beats.per.minute', 'tempo_class', 'feeling']])
else:
    st.warning("No results found based on your criteria.")

# Show overall stats only after the search or selectbox is applied
st.markdown("### 🔍 Tempo Distribution")
tempo_counts = filtered_df['tempo_class'].value_counts()
st.bar_chart(tempo_counts)

st.markdown("### 🧠 Feeling Distribution")
feeling_counts = filtered_df['feeling'].value_counts()
st.bar_chart(feeling_counts)

# Optional visual analysis
st.markdown("### 📊 Feature Comparison (Energy, Danceability, Valance)")

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
sns.boxplot(data=filtered_df, x='tempo_class', y='energy', ax=axes[0]).set(title='Energy by Tempo')
sns.boxplot(data=filtered_df, x='tempo_class', y='danceability', ax=axes[1]).set(title='Danceability by Tempo')
sns.boxplot(data=filtered_df, x='tempo_class', y='valance', ax=axes[2]).set(title='Valance by Tempo')
st.pyplot(fig)

# Footer
st.caption("Project Objective: Classify songs by tempo, analyze characteristics, and explore emotional impact.")


Writing app.py


In [ ]:
!ngrok config add-authtoken 2wQlEIuIdfduPUTTioz0dYYrT1d_3eBwKe7NKT8Spo88bYuLh


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

ngrok.kill()  # Kill any previous tunnels

# Start a new HTTP tunnel to port 8501
public_url = ngrok.connect(addr="8501", proto="http")
print("Streamlit app URL:", public_url)

# Run Streamlit
!streamlit run app.py &> /dev/null &


Streamlit app URL: NgrokTunnel: "https://85f8-34-147-37-128.ngrok-free.app" -> "http://localhost:8501"


🔍 Current Colab Runtime Type: CPU
⏱️ Time taken to determine runtime: 0.001338 seconds
